# Importing Libraries

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style
%matplotlib inline

# Read the Data as Dataframe

In [2]:
# Reading Data
df_online_retail = pd.read_csv('../00_Data/data.csv', encoding= 'unicode_escape', na_values='nan')

# Initial Check of Data:
Check of shape, data types, missing values and initial statics of the columns

In [17]:
# Check shape of data 
print(df_online_retail.shape)
print("The dataframe consists of", df_online_retail.shape[1], "columns and of", df_online_retail.shape[0], "rows")

(541909, 8)
The dataframe consists of 8 columns and of 541909 rows


In [16]:
# Check first five rows of Data
df_online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
# Check Data Types
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [19]:
# Check number of missing values per column
df_online_retail.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

In [23]:
# Descriptive Statistics 
df_online_retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


There seem to be orders with a negative quantity. These orders should be dropped from further analysis. 

**Potential Changes for Data**
- Drop all Missing Values
- Drop Entries with Negative Quantity
- Change Datetime Type & CustomerID to String
- Use Label Encoder for StockCode & CustomerID

# Data Cleaning & Transformation

In [35]:
# Drop missing values
df_online_retail = df_online_retail.dropna()

# Sanity Check 
df_online_retail.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [36]:
# Drop Entries with Negative Quantity
df_online_retail = df_online_retail[df_online_retail['Quantity']>0]

# Sanity Check 
df_online_retail.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


In [37]:
# Data Type Transformations

# Convert CustomerID to String
df_online_retail['CustomerID'] = df_online_retail['CustomerID'].astype(str)

# Convert Invoice Date into Date time
df_online_retail['InvoiceDate'] = pd.to_datetime(df_online_retail['InvoiceDate'])

# Sanity Check
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  object        
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.3+ MB


In [38]:
# User & Item  Label Encoder 

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Set up Objects for User and Item Column
le1 = LabelEncoder()
le2 = LabelEncoder()

# Fit the encoders to the columns in question
le1.fit(df_online_retail['StockCode'])
le2.fit(df_online_retail['CustomerID'])

# Transform dataframe with encoded labels
df_online_retail['StockCode'] = le1.transform(df_online_retail['StockCode'])
df_online_retail['CustomerID'] = le2.transform(df_online_retail['CustomerID'])

# Check Change 
df_online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,3233,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,4017,United Kingdom
1,536365,2643,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,4017,United Kingdom
2,536365,2847,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,4017,United Kingdom
3,536365,2795,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,4017,United Kingdom
4,536365,2794,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,4017,United Kingdom


In [42]:
# Check Impact of Data Cleaning on Shape
initial_number_rows = 541909
print("Shape of transformed Dataframe:", df_online_retail.shape)
print(initial_number_rows - df_online_retail.shape[0], "rows were removed from the Dataframe during data cleaning.")


Shape of transformed Dataframe: (397924, 8)
143985 rows were removed from the Dataframe during data cleaning.
